# レコメンドシステム
- Jester Datasetを使ってユーザにジョークをお薦めするシステムを実装
- 今回実装したレコメンド手法
    - コンテンツベースフィルタリング
        - LSA(Latent Semantic Analysis)
            - SVD(sigular value decomposition)
    - 協調フィルタリング
        - ユーザベース
        - 標準化（中央化）

In [3]:
!./download.sh

In [1]:
import pandas as pd
import numpy as np
import os
import math
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize

# データセットの説明
- 英語のジョーク集の評価を集めたJester Collaborative Filtering Datasetを使用
- これは150種類のジョークに対して50692人が-10〜+10で評価している

しかし，[本家サイト](http://eigentaste.berkeley.edu/dataset/)のは評価値がxlsファイルだったり、ジョーク本文をhtmlからパースする必要があったりするため、非常に扱い

そこで，csv/tsvファイルに加工されて[kaggleで提供されているデータセット](https://www.kaggle.com/crawford/jester-online-joke-recommender)があるので，そこから jesterfinal151cols.csv (評価値) と jester_items.tsv (ジョーク本文)をダウンロード

Note:
kaggleのjester_items.tsvは150番目の以下のジョークが抜けているので追加しなければいけない

150: In an interview with David Letterman, Carter passed along an anecdote of a translation problem in Japan. Carter was speaking at a business lunch in Tokyo, where he decided to open his speech with a brief joke. He told the joke, then waited for the translator to announce the Japanese version. Even though the story was quite short, Carter was surprised by how quickly the interpreter was able to re-tell it. Even more impressive was the reaction from the crowd. Carter thought the story was cute, but not outright hilarious, yet the crowd broke right up. Carter was very flattered. After the speech, Carter wanted to meet the translator to ask him how he told the joke. Perhaps there is better way to tell the joke? When Carter asked how the joke had been told in Japanese, the translator responded, "I told them, 'President Carter has told a very funny joke. Please laugh now.'"

In [22]:
# まずはジョーク本文の読み込み
jokes = pd.read_csv("dataset/jester_items.tsv", sep='\t', names=['id', 'joke'])
# column"id"は不要なので削除
jokes.drop("id", axis=1, inplace=True)

# 150番目のjokeを追加
jokes.loc[jokes.shape[0]] = "In an interview with David Letterman, Carter passed along an anecdote of a translation problem in Japan. Carter was speaking at a business lunch in Tokyo, where he decided to open his speech with a brief joke. He told the joke, then waited for the translator to announce the Japanese version. Even though the story was quite short, Carter was surprised by how quickly the interpreter was able to re-tell it. Even more impressive was the reaction from the crowd. Carter thought the story was cute, but not outright hilarious, yet the crowd broke right up. Carter was very flattered. After the speech, Carter wanted to meet the translator to ask him how he told the joke. Perhaps there is better way to tell the joke? When Carter asked how the joke had been told in Japanese, the translator responded, I told them, President Carter has told a very funny joke. Please laugh now."

# dataframeがどういう風になっているか
print("shape: \n\t", jokes.shape)
print("columns: \n\t", ",".join(jokes.columns), "\n")
print(jokes.head(10), "\n")
print(jokes.tail(10), "\n")

# いくつかjokeを見てみる
print(jokes["joke"][0])
print("(男が医者を訪ねます。 医師は、「あなたに悪い知らせがあります。あなたは癌とアルツハイマー病を患っています」と言います。 男は「まあ、がんにかかっていないことを神に感謝します！」と答えます。)")
print()
print(jokes["joke"][145])
print("(アメリカ人の他国の働き方に対する皮肉かな)")

shape: 
	 (150, 1)
columns: 
	 joke 

                                                joke
0  A man visits the doctor. The doctor says, "I h...
1  This couple had an excellent relationship goin...
2  Q. What's 200 feet long and has 4 teeth? A. Th...
3  Q. What's the difference between a man and a t...
4  Q. What's O. J. Simpson's web address? A. Slas...
5  Bill and Hillary Clinton are on a trip back to...
6  How many feminists does it take to screw in a ...
7  Q. Did you hear about the dyslexic devil worsh...
8  A country guy goes into a city bar that has a ...
9  Two cannibals are eating a clown. One turns to... 

                                                  joke
140  Jack Bauer can get McDonald's breakfast after ...
141  One day, three men went to a shrine to ask the...
142  A preist, a 12-year-old kid, and the smartest ...
143  A man is driving in the country one evening wh...
144  A blonde, brunette, and a red head are all lin...
145  America: 8:00 - Welcome to work! 12:00 - L

# 1. コンテンツベースフィルタリング
アイテムそのものの特徴を利用して, 推薦したいユーザがこれまで高評価したアイテムと類似するアイテムを推薦する方法

過去に「ハリー・ポッターと賢者の石」と「指輪物語 1」を高評価しているユーザは, おそらく「小説」「ファンタジー」を嗜好しているため, 「ナルニア国物語 1」などを薦める, といった感じ

- 長所
    - まだ評価されていないアイテムでもアイテムの特徴によってレコメンド可能
    - システム全体のユーザ数・評価数が少ない状況でも予測精度は下がらない
- 短所
    - メタデータの量が十分に無い場合は推薦されるアイテムの質が安定しない
    - 他人の評価データを利用できない

### TF-IDFで特徴ベクトルを抽出
- TF(Term Frequency)
    - 「単語の出現頻度」
    - 各文書においてその単語がどのくらい出現したのかを意味し，よく出現する単語は，その文書の特徴を判別するのに有用
    
    $TF = \frac{文書Aにおける単語Xの出現頻度}{文書Aにおける全単語の出現頻度の和}$<br /><br /><br />
- IDF(Inverse Document Frequency)
    - 「逆文書頻度」
    - 単語が「レア」なら高い値を,「色々な文書によく出現する単語」なら低い値を示すもので,レアな単語はその文書の特徴を判別するのに有用
    
    $IDF = \log \frac{全文書数}{単語Xを含む文書数}$<br /><br /><br />
    
- TF-IDF
    - 上記の2つを組み合わせ
    - レアな単語が何回も出てくるようなら,文書を分類する際にその単語の重要度を上げるというもの

    $\begin{align}
    TFIDF &= TF \times IDF \\
    &= (単語の出現頻度) \times (各単語のレア度)
    \end{align}$

In [23]:
# 実際にTF-IDFで特徴ベクトルを出してみる
vector = TfidfVectorizer(norm='l2', min_df=1)
x = vector.fit_transform(jokes['joke'])
pd.DataFrame(x.toarray(), columns=vector.get_feature_names())

,00,000,10,100,1040,11,12,125,13,14,...,yesterday,yet,york,you,young,younger,your,yourself,zipper,zo
0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.140357,0.000000,0.0000,0.000000,0.0,0.0000,0.0
1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.096651,0.000000,0.0000,0.000000,0.0,0.0000,0.0
2,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.0000,0.0
3,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.196877,0.000000,0.0000,0.000000,0.0,0.0000,0.0
4,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.0000,0.0
5,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.123529,0.000000,0.0000,0.000000,0.0,0.0000,0.0
6,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.0000,0.0
7,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.106017,0.000000,0.0000,0.000000,0.0,0.0000,0.0
8,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.075814,0.000000,0.0000,0.000000,0.0,0.0000,0.0
9,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.109107,0.000000,0.0000,0.000000,0.0,0.0000,0.0


In [24]:
'''
BoWベクトル同士のコサイン類似度行列を計算
words_vector : 特徴ベクトル(type:ndarray)
'''
def get_similarities(words_vector):
    similarities = np.zeros((words_vector.shape[0], words_vector.shape[0]))
    
    for i1, v1 in enumerate(words_vector):
        for i2, v2 in enumerate(words_vector):
            if i1 == i2:
                similarities[i1][i2] = 0
            else:
                similarities[i1][i2] = np.dot(v1, v2.T) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    return similarities
# これはlinear_kernelで一発ですね

similarities = get_similarities(vector.transform(jokes['joke']).toarray())
similarities[:5, :5]

array([[0.        , 0.04904041, 0.01627974, 0.08054505, 0.        ],
       [0.04904041, 0.        , 0.01088568, 0.02954297, 0.00465303],
       [0.01627974, 0.01088568, 0.        , 0.02738159, 0.00981273],
       [0.08054505, 0.02954297, 0.02738159, 0.        , 0.00947818],
       [0.        , 0.00465303, 0.00981273, 0.00947818, 0.        ]])

In [25]:
# 類似度が一番大きいジョークを探してみる
print("類似度最大 : ", np.where(similarities==np.max(similarities))[0], "\n")
print(jokes["joke"][112], "\n")
print(jokes["joke"][132], "\n")

類似度最大 :  [112 132] 

The new employee stood before the paper shredder looking confused. "Need some help?" a secretary asked. "Yes," he replied. "How does this thing work?" "Simple," she said, taking the fat report from his hand and feeding it into the shredder. "Thanks, but where do the copies come out?" 

The new employee stood before the paper shredder looking confused. "Need some help?" a secretary, walking by, asked. "Yes," he replied, "how does this thing work?" "Simple," she said, taking the fat report from his hand and feeding it into the shredder. "Thanks, but where do the copies come out?" 



違いは　”walking by” があるかないかだけでした

### データの整形　＆　trainデータセットとtestデータセットの作成

In [26]:
# まずはジョークへの評価値が入ったデータの読み込み
# 評価値は-10〜+10の連続値となっており, 未評価は99
ratings = np.genfromtxt('dataset/jesterfinal151cols.csv', delimiter=',', filling_values=(99))
ratings = np.delete(ratings, 0, axis=1)

print(ratings.shape)
ratings

(50692, 150)


array([[99., 99., 99., ..., 99., 99., 99.],
       [99., 99., 99., ..., 99., 99., 99.],
       [99., 99., 99., ..., 99., 99., 99.],
       ...,
       [99., 99., 99., ..., 99., 99., 99.],
       [99., 99., 99., ..., 99., 99., 99.],
       [99., 99., 99., ..., 99., 99., 99.]])

- 後の処理（LSI）で未評価値が99点になっていると，正しく評価値の予測ができない<br>
- そのため，ここでは未評価値を0点に<br>
- 未評価値の事前の補完は，レコメンド性能を向上させる上で色々試作しなければいけないところではあるが．．．

In [27]:
# 行列内で0が入っている割合を確認
print("0点が入っている割合 : {:.2f}".format(len(np.where(ratings==0)[0]) / (ratings.shape[0] * ratings.shape[1]) * 100), "%", "\n")

# 未評価値　99点　-> 0点
ratings[ratings==99] = 0

# こうすると1つも評価していないという人が現れるのでその人を除外
removed_user = np.where(np.all(ratings==0, axis=1)==True)[0]
ratings = np.delete(ratings, removed_user, axis=0)
print("1つもjokeを評価していないと判定されてしまったuser : ", removed_user)
print(ratings.shape, "\n")

# さらにtrainデータを作る際に邪魔になるのと，計算量を減らすために評価数の少ない人を削除
removed_user = np.where(np.count_nonzero(ratings, axis=1)<=20)[0]
ratings = np.delete(ratings, removed_user, axis=0)
print("評価数の少ないuser : ", removed_user)
print(ratings.shape)

0点が入っている割合 : 0.05 % 

1つもjokeを評価していないと判定されてしまったuser :  [1465]
(50691, 150) 

評価数の少ないuser :  [    2     7    10 ... 50687 50688 50690]
(25045, 150)


In [28]:
'''
trainデータとtestデータを作成
matrix : train, testに分割したい行列
'''
def make_train_test(matrix):
    np.random.seed(1)
    blank_num = 5
    train_matrix = matrix.copy()
    blank_flag_matrix = np.zeros(matrix.shape) # 意図的に欠損値とした位置
    blank_flag_matrix_high_rate = np.zeros(matrix.shape) # 意図的に欠損値とした位置の中で値が5以上の位置
    
    for i in range(len(matrix)):
        candidate = np.where(matrix[i] != 0)
        candidate = list(candidate[0])
        blank_position = np.random.choice(candidate, size=blank_num, replace=False)
        
        for j in blank_position:
            if train_matrix[i][j] >= 5:
                blank_flag_matrix_high_rate[i][j] = 1
                
            train_matrix[i][j] = 0
            blank_flag_matrix[i][j] = 1
            
    return train_matrix, blank_flag_matrix, blank_flag_matrix_high_rate

In [29]:
train_matrix, blank_flag_matrix, blank_flag_matrix_high_rate = make_train_test(ratings)
test_matrix = ratings * blank_flag_matrix
print(train_matrix.shape)
print(test_matrix.shape)

# train_matrix と test_matrixの例
# user23の全10つのジョークへの評価に関して
#     (ratings)[3, 6, -9, 1, 2, -4, -1, 6, 9, -2]  -> (train_matrix)           [3, 6, 0, 0, 2, 0, 0, 6, 9, 0]
#                                                                          -> (blank_flag_matrix) [0, 0, 1, 1, 0, 1, 1, 0, 0, 1]

(25045, 150)
(25045, 150)


In [30]:
# 未評価の割合
print("元データ : {:.2f}".format(len(np.where(ratings==0)[0]) / (ratings.shape[0] * ratings.shape[1]) * 100), "%")
print("加工後 : {:.2f}".format(len(np.where(train_matrix==0)[0]) / (train_matrix.shape[0] * train_matrix.shape[1]) * 100), "%")

元データ : 62.29 %
加工後 : 65.62 %


### 欠損部分の予測

In [31]:
'''
未評価jokeの評価値予測
rating          : あるユーザの全ジョークへの評価値
item_index : 評価を予測したい未評価ジョークの位置
similarities  : 類似度が入った行列
'''
def calc_predict_rating(rating, item_index, similarities):
    times_total = 0
    sim_total = 0
    
    for i, r in enumerate(rating):
        if r != 0:
            # 加重平均
            times_total += similarities[item_index][i] * r
            sim_total += similarities[item_index][i]
            
    if sim_total != 0:
        return times_total / sim_total
    else:
        return np.nan

In [32]:
# ちゃんと動くかも兼ねて見てみる
# user0 の　joke0　への評価予測
print("user0 の　joke0　への予測評価 : {:.2f}".format(calc_predict_rating(train_matrix[0], 0, similarities)), "点")
# user0 の　joke4　への評価予測
print("user0 の　joke4　への予測評価 : {:.2f}".format(calc_predict_rating(train_matrix[0], 4, similarities)), "点")

user0 の　joke0　への予測評価 : 2.97 点
user0 の　joke4　への予測評価 : -1.26 点


In [33]:
'''
関数calc_predict_ratingを全要素に対して処理を実行
ratings 　　　　　　: 全ユーザの全ジョークへの評価が入った行列
similarities : 類似度が入った行列
'''
def get_all_predict_rating(ratings, similarities):
    pred = ratings.copy()
    
    for i in range(ratings.shape[0]):
        for j in range(ratings.shape[1]):
            if ratings[i][j] == 0:
                temp = calc_predict_rating(ratings[i], j, similarities)
                pred[i][j] = temp
            else:
                pred[i][j] = ratings[i][j]
            
    return pred

In [34]:
pred = get_all_predict_rating(train_matrix, similarities) # 結構時間かかります
pred

array([[ 2.97090162,  3.78289316,  3.36670229, ...,  4.10358199,
         4.12438427,  4.17685516],
       [ 3.30352052,  3.84278975,  3.01236077, ...,  3.42563589,
         4.68920834,  4.24142947],
       [ 3.00456941,  2.51949843,  2.73518575, ...,  2.84938026,
         2.59017054,  2.80334784],
       ...,
       [-0.2146722 ,  0.80796821, -1.40717422, ...,  0.19136005,
         2.04135457,  0.43029211],
       [ 2.50524035,  2.39888517,  2.55725687, ...,  2.55776209,
         2.22446009,  2.46875   ],
       [ 0.59422325,  0.57911172,  0.89051904, ...,  0.93443361,
         0.34910727,  0.78633267]])

### 評価値の予測制度の評価
- RMSE（root mean squared error）を使用
- 実測値と予測値が近づくほど,RMSE は小さくなる.逆に,実測値と予測値が遠くなると,RMSE が著しく増える
    - そのため、外れ値が含まれると,実測値と予測値の差が大きく離れるため,RMSE が著しく大きくなる
    - このことから,RMSE は外れ値の影響を受けやすいといわれている
<br><br><br>
\begin{align}
RMSE = \sqrt{\frac{1}{n}\sum_{k=1}^{n}(\hat{r}_i - r_i)^2}
\end{align}

In [35]:
'''
RMSEを算出
train : 予測評価が行われたジョークのうち，本来はデータが入っていた箇所だけに値が入った行列を作成
test  : trainの本来の評価値
'''
def calc_RMSE(train, test):
    diff = train - test
    power = diff * diff
    power_sum = np.nansum(power, axis=0)
    power_sum = np.sum(power_sum)
    
    return math.sqrt(power_sum / (train.shape[0] * train.shape[1]))

In [36]:
train = pred * blank_flag_matrix
test = test_matrix.copy()

rmse = calc_RMSE(train, test)
print("RMSE : {:.4f}".format(rmse))

'''
RMSE : 0.8174
'''

RMSE : 0.8174


'\nRMSE : 0.8174\n'

# 1.2 LSA(Latent Semantic Analysis)
- 日本語だと「潜在意味解析」
- SVDで求まった特異値を影響度の少ない順に削っていくことによって次元を削減していくもの
- その後，次元削減された特異値の行列を用いて類似度を算出
- SVDとは
    - $A=UΣV^T$
    - Σ
        - Aの特異値（σ0, σ1, σ2, ...）を降順にソートし，対角行列にしたもの
        - Uの対応する列の「重要度」みたいなものを表現
        - 例えばσ2を考えた時，これに対応するU[:, 3]との掛け算は，U[:, 3]のAを表す時の重要度を示す

In [37]:
joke_tfidf_matrix = vector.transform(jokes['joke']).toarray()
U, s, Vt = np.linalg.svd(joke_tfidf_matrix, full_matrices=False) # train_matrixを特異値分解

In [38]:
print(U[:5, :5])
print(U.shape)

[[-0.07276221 -0.08063344  0.27927387 -0.10780873  0.06444291]
 [-0.09444847 -0.00876188 -0.1385658  -0.16712446 -0.06072223]
 [-0.03264647 -0.01959569 -0.00340944  0.04541486  0.01829848]
 [-0.05547295 -0.02672117  0.15324105  0.0989765   0.14544937]
 [-0.00269359 -0.00394613  0.0083797   0.01603512  0.01278774]]
(150, 150)


In [39]:
print(s.shape) # ｓの形が潰れてしまっている対角化
s = np.diag(s)
print(s[:5, :5])
print(s.shape)

(150,)
[[3.43307102 0.         0.         0.         0.        ]
 [0.         1.72201463 0.         0.         0.        ]
 [0.         0.         1.47291643 0.         0.        ]
 [0.         0.         0.         1.4403313  0.        ]
 [0.         0.         0.         0.         1.41063272]]
(150, 150)


In [40]:
print(Vt[:5, :5])
print(Vt.shape)

[[-0.01334151 -0.01012817 -0.01282941 -0.0058884  -0.00428306]
 [-0.00798765 -0.00608345 -0.00887612 -0.00126697 -0.00022067]
 [ 0.00912368 -0.00151069  0.01656942 -0.00776304  0.00907243]
 [-0.03058032  0.00953875 -0.0065837   0.0092565  -0.00945583]
 [ 0.06519652  0.00628721 -0.01104715 -0.00183755 -0.0155897 ]]
(150, 2216)


In [41]:
'''
指定された特異値の分だけ次元削減
k 　　　　　　　　　　: 残した特異値の個数
U, s, Vt : 特異値分解で得られた行列たち
'''
def calc_ajusted_svd(k, U, s, Vt):
    return np.dot(np.dot(U[:, :k], s[:k, :k]), Vt[:k, :])
    #return np.dot(U[:, :k], s[:k, :k])

In [ ]:
# ハイパーパラメータｋを調節
min_k = 5

for k in range(min_k, s.shape[0], 5):
    adjusted = calc_ajusted_svd(s.shape[0] - k, U, s, Vt)
    similarities_svd = get_similarities(adjusted)
    pred_svd = get_all_predict_rating(train_matrix, similarities_svd) # 結構時間かかります
    
    train_svd = pred_svd * blank_flag_matrix
    test_svd = test_matrix
    
    rmse_svd = calc_RMSE(train_svd, test_svd)
    print("削除する特異値の個数 : ", k, "\tRMSE : {:.4f}".format(rmse_svd))
    
'''
削除する特異値の個数 :  5 	RMSE : 0.8177
削除する特異値の個数 :  10 	RMSE : 0.8179
削除する特異値の個数 :  15 	RMSE : 0.8189
削除する特異値の個数 :  20 	RMSE : 0.8185
削除する特異値の個数 :  25 	RMSE : 0.8188
削除する特異値の個数 :  30 	RMSE : 0.8190
削除する特異値の個数 :  35 	RMSE : 0.8193
削除する特異値の個数 :  40 	RMSE : 0.8204
削除する特異値の個数 :  45 	RMSE : 0.8210
削除する特異値の個数 :  50 	RMSE : 0.8192
削除する特異値の個数 :  55 	RMSE : 0.8890
削除する特異値の個数 :  60 	RMSE : 0.8210
削除する特異値の個数 :  65 	RMSE : 0.9724
削除する特異値の個数 :  70 	RMSE : 0.9121
削除する特異値の個数 :  75 	RMSE : 1.0908
削除する特異値の個数 :  80 	RMSE : 0.8480
削除する特異値の個数 :  85 	RMSE : 0.9927
削除する特異値の個数 :  90 	RMSE : 1.1184
削除する特異値の個数 :  95 	RMSE : 3.4013
削除する特異値の個数 :  100 	RMSE : 0.8993
削除する特異値の個数 :  105 	RMSE : 0.8909
削除する特異値の個数 :  110 	RMSE : 0.8391
削除する特異値の個数 :  115 	RMSE : 724.0293
削除する特異値の個数 :  120 	RMSE : 0.8909
削除する特異値の個数 :  125 	RMSE : 1.2015
削除する特異値の個数 :  130 	RMSE : 0.8607
削除する特異値の個数 :  135 	RMSE : 0.8176 <- 最小ではあるが．．．
削除する特異値の個数 :  140 	RMSE : 0.8191
削除する特異値の個数 :  145 	RMSE : 0.8210
'''

SVD前の精度 = RMSE : 0.8174

**上のやつがうまくいかない理由を考察（vranoさんのを聞いていて思いついた）**<br>
例えば，car, automobile をそれぞれ持つ文書があるとする<br>
期待することは，SVDの次元削減により　car, automobileが似たようなものだと判断されること<br>
(これらの語の前に同じ単語（drive）が来ることが多いから（共起性）)<br>
しかし今はステミングしていないので，　drive，drives，droveを互いに区別してしまったためと思われる

In [24]:
# 上の考察があっているか確かめるためにステミングしてみます
# ステミングに必要なのでダウンロードさせる
# やらないとエラーします
nltk.download('punkt')

class PorterTokenizer(object):
    def __init__(self):
        self.porter = PorterStemmer()
        
    def __call__(self, doc):
        return [self.porter.stem(w) for w in word_tokenize(doc)]
    

# Porterでステミングして、TF-IDFで特徴ベクトルを構築
vector_stem = TfidfVectorizer(norm='l2', min_df=1, tokenizer=PorterTokenizer())
x = vector_stem.fit_transform(jokes['joke'])
stem_jokes = pd.DataFrame(x.toarray(), columns=vector_stem.get_feature_names())
stem_jokes

[nltk_data] Downloading package punkt to /Users/shoji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,!,$,%,','','bow,'celebr,'congratul,'d,'holi,...,yesterday,yet,york,you,young,younger,your,yourself,zipper,zo
0,0.084571,0.000000,0.0,0.000000,0.117021,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.136105,0.00000,0.00000,0.000000,0.0,0.000000,0.0
1,0.000000,0.000000,0.0,0.000000,0.121192,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.093971,0.00000,0.00000,0.000000,0.0,0.000000,0.0
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.189602,0.00000,0.00000,0.000000,0.0,0.000000,0.0
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0
5,0.075552,0.000000,0.0,0.000000,0.052271,0.000000,0.0,0.0,0.10278,0.0,...,0.000000,0.000000,0.0,0.121590,0.00000,0.00000,0.000000,0.0,0.000000,0.0
6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0
7,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.099212,0.00000,0.00000,0.000000,0.0,0.000000,0.0
8,0.086617,0.000000,0.0,0.160207,0.029963,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.069699,0.00000,0.00000,0.000000,0.0,0.000000,0.0
9,0.000000,0.000000,0.0,0.000000,0.095059,0.000000,0.0,0.0,0.00000,0.0,...,0.000000,0.000000,0.0,0.110562,0.00000,0.00000,0.000000,0.0,0.000000,0.0


In [25]:
joke_tfidf_matrix = vector_stem.transform(jokes['joke']).toarray()

similaritiy_stem = get_similarities(joke_tfidf_matrix)
similaritiy_stem[:5, :5]

array([[0.        , 0.11172702, 0.03472513, 0.11987884, 0.05389235],
       [0.11172702, 0.        , 0.03097284, 0.06004603, 0.02763212],
       [0.03472513, 0.03097284, 0.        , 0.17293104, 0.09896859],
       [0.11987884, 0.06004603, 0.17293104, 0.        , 0.10458286],
       [0.05389235, 0.02763212, 0.09896859, 0.10458286, 0.        ]])

In [ ]:
pred_stem = get_all_predict_rating(train_matrix, similaritiy_stem) # 結構時間かかります

train = pred_stem * blank_flag_matrix

rmse = calc_RMSE(train, test)
print("RMSE : {:.4f}".format(rmse))

'''
RMSE : 0.8081
'''
'''
ステミングしなかったとき
RMSE : 0.8174
'''

In [26]:
U, s, Vt = np.linalg.svd(joke_tfidf_matrix, full_matrices=False)
s = np.diag(s)

In [27]:
'''
SVDで次元削減した行列を用いた欠損予測とその評価
'''
def svd_reduce_dim(use_only_high_rate=False):
    # ハイパーパラメータｋを調節
    min_k = 5
    
    for k in range(min_k, s.shape[0], 5):
        adjusted = calc_ajusted_svd(s.shape[0] - k, U, s, Vt)
        similarities_svd = get_similarities(adjusted)
        pred_svd = get_all_predict_rating(train_matrix, similarities_svd) # 結構時間かかります
    
        train = pred_svd * blank_flag_matrix
        test = test_matrix.copy()
        
        if use_only_high_rate == True:
            train *= blank_flag_matrix_high_rate
            test *= blank_flag_matrix_high_rate
    
        rmse = calc_RMSE(train, test)
        print("削除する特異値の個数 : ", k, "\tRMSE : {:.4f}".format(rmse))

In [ ]:
svd_reduce_dim()
    
'''
削除する特異値の個数 :  5 	RMSE : 0.8083
削除する特異値の個数 :  10 	RMSE : 0.8082
削除する特異値の個数 :  15 	RMSE : 0.8081
削除する特異値の個数 :  20 	RMSE : 0.8085
削除する特異値の個数 :  25 	RMSE : 0.8090
削除する特異値の個数 :  30 	RMSE : 0.8089
削除する特異値の個数 :  35 	RMSE : 0.8092
削除する特異値の個数 :  40 	RMSE : 0.8087
削除する特異値の個数 :  45 	RMSE : 0.8082
削除する特異値の個数 :  50 	RMSE : 0.8078
削除する特異値の個数 :  55 	RMSE : 0.8080
削除する特異値の個数 :  60 	RMSE : 0.8075
削除する特異値の個数 :  65 	RMSE : 0.8069
削除する特異値の個数 :  70 	RMSE : 0.8070
削除する特異値の個数 :  75 	RMSE : 0.8076
削除する特異値の個数 :  80 	RMSE : 0.8073
削除する特異値の個数 :  85 	RMSE : 0.8077
削除する特異値の個数 :  90 	RMSE : 0.8088
削除する特異値の個数 :  95 	RMSE : 0.8084
削除する特異値の個数 :  100 	RMSE : 0.8083
削除する特異値の個数 :  105 	RMSE : 0.8080
削除する特異値の個数 :  110 	RMSE : 0.8076
削除する特異値の個数 :  115 	RMSE : 0.8077
削除する特異値の個数 :  120 	RMSE : 0.8073
削除する特異値の個数 :  125 	RMSE : 0.8067　<- 最小ではあるが...
削除する特異値の個数 :  130 	RMSE : 0.8084
削除する特異値の個数 :  135 	RMSE : 0.8071
削除する特異値の個数 :  140 	RMSE : 0.8090
削除する特異値の個数 :  145 	RMSE : 0.8112
'''

SVD前の精度 = RMSE : 0.8081

予測評価の高いものをレコメンドするのが基本<br>
なので，RMSEを出すときにtest_matrixで高評価（5点以上くらい？）のものだけで予測評価を出してみるのがいいかも

In [28]:
# SVDを使う前のhigh rateの予測精度
train = pred * blank_flag_matrix_high_rate
test = test_matrix * blank_flag_matrix_high_rate

rmse = calc_RMSE(train, test)
print("RMSE : {:.4f}".format(rmse))

'''
RMSE : 0.4311
'''

RMSE : 0.4311


'\nRMSE : 0.4311\n'

In [ ]:
svd_reduce_dim(True)
'''
削除する特異値の個数 :  5 	RMSE : 0.4296
削除する特異値の個数 :  10 	RMSE : 0.4295
削除する特異値の個数 :  15 	RMSE : 0.4295
削除する特異値の個数 :  20 	RMSE : 0.4293
削除する特異値の個数 :  25 	RMSE : 0.4293
削除する特異値の個数 :  30 	RMSE : 0.4289
削除する特異値の個数 :  35 	RMSE : 0.4287
削除する特異値の個数 :  40 	RMSE : 0.4285
削除する特異値の個数 :  45 	RMSE : 0.4282
削除する特異値の個数 :  50 	RMSE : 0.4283
削除する特異値の個数 :  55 	RMSE : 0.4286
削除する特異値の個数 :  60 	RMSE : 0.4280
削除する特異値の個数 :  65 	RMSE : 0.4277
削除する特異値の個数 :  70 	RMSE : 0.4275
削除する特異値の個数 :  75 	RMSE : 0.4275
削除する特異値の個数 :  80 	RMSE : 0.4276
削除する特異値の個数 :  85 	RMSE : 0.4275
削除する特異値の個数 :  90 	RMSE : 0.4276
削除する特異値の個数 :  95 	RMSE : 0.4274 <- 最小
削除する特異値の個数 :  100 	RMSE : 0.4276
削除する特異値の個数 :  105 	RMSE : 0.4277
削除する特異値の個数 :  110 	RMSE : 0.4277
削除する特異値の個数 :  115 	RMSE : 0.4290
削除する特異値の個数 :  120 	RMSE : 0.4288
削除する特異値の個数 :  125 	RMSE : 0.4295
削除する特異値の個数 :  130 	RMSE : 0.4298
削除する特異値の個数 :  135 	RMSE : 0.4294
削除する特異値の個数 :  140 	RMSE : 0.4327
削除する特異値の個数 :  145 	RMSE : 0.4376
'''

# 2. 協調フィルタリング
- ユーザベース
    - 「あるユーザは未評価アイテムに対して,当該ユーザと似たような嗜好をしている他ユーザと同じような評価をするだろう」という仮定に基づいたもの
- アイテムベース
    - 「アイテム同士の類似度とあるユーザの過去に評価したアイテムの評価点を用いて未評価アイテムの評価点を予測する」というもの
   
<br>
今回は「ユーザベース」のみを実装(中心化を行う上で，アイテムベースだと上手く行かなかったから)<br>
類似度の算出にはコサイン類似度を使用

In [44]:
'''
コサイン類似度を算出
matrix : 縦軸 -> ユーザ, 横軸 -> 各ジョークへの評価 の行列
'''
def cosine_similarity_matrix(matrix):
    epsilon=1e-9
    norm = np.linalg.norm(matrix, ord=2, axis=0) + epsilon # 各列ベクトルのノルムを算出
    normalized = matrix / norm # 各々の列ベクトルを正規化
    
    return np.dot(normalized, normalized.T)

In [45]:
# ユーザ間の類似度行列を作成
similarity_matrix = cosine_similarity_matrix(train_matrix)
np.fill_diagonal(similarity_matrix, 0) # 対角を0に

print(similarity_matrix.shape)
similarity_matrix

(25045, 25045)


array([[ 0.00000000e+00,  2.19193146e-02, -3.53921202e-03, ...,
         4.26696049e-05,  2.07556983e-04,  3.79347717e-04],
       [ 2.19193146e-02,  0.00000000e+00, -9.62085321e-03, ...,
        -2.45748390e-05,  2.50024402e-04, -4.89254944e-05],
       [-3.53921202e-03, -9.62085321e-03,  0.00000000e+00, ...,
         7.48778139e-05,  1.40171201e-04,  1.89562615e-05],
       ...,
       [ 4.26696049e-05, -2.45748390e-05,  7.48778139e-05, ...,
         0.00000000e+00, -7.14696371e-05, -2.81083305e-05],
       [ 2.07556983e-04,  2.50024402e-04,  1.40171201e-04, ...,
        -7.14696371e-05,  0.00000000e+00, -5.34257599e-06],
       [ 3.79347717e-04, -4.89254944e-05,  1.89562615e-05, ...,
        -2.81083305e-05, -5.34257599e-06,  0.00000000e+00]])

In [46]:
'''
未評価ジョークの評価値を予測
user_id 　　　: 予測を行いたいユーザのID
similarity 　: 各ジョーク間の類似度を表す行列
mean 　　　　　　: 各ユーザの平均値のリスト（標準化で使用）
'''
def get_pred_ratings(user_id, similarity, mean=0, top_k=10):
    user_index = user_id - 1
    epsilon=1e-9
    pred_rating_specific_user = train_matrix[user_index, :].copy()
    
    # 各userに対し，類似度top-Kのuserを抽出
    top_k_users = [np.argsort(similarity[:, user_index])[:-top_k-1:-1]]
    
    for i in range(pred_rating_specific_user.shape[0]):
        if pred_rating_specific_user[i] == 0:
            # 類似度による重み付き平均
            pred_rating_specific_user[i] = similarity[i, :][top_k_users].dot(ratings[:, i][top_k_users])
            pred_rating_specific_user[i] /= np.sum(np.abs(similarity[i, :][top_k_users]))
    
    # 中心化で使用する処理
    # 中心化で減らされた平均値（mean）を足して，元に戻す
    if mean != 0:
        pred_rating_specific_user += mean[user_index]
    
    # ユーザが既に評価したアイテムのスコアはゼロに直す
    pred_rating_specific_user = pred_rating_specific_user * blank_flag_matrix[user_index, :]
    
    return pred_rating_specific_user

'''
予測結果を表示
'''
def show_results(user_id, predict_rating):
    user_index = user_id - 1
    
    for i in np.nonzero(predict_rating)[0]:
        print("ジョーク", i)
        print("\t真値 :\t", test_matrix[user_index][i])
        print("\t予測値 : \t", predict_rating[i])

In [47]:
# user_ID 19の人の予測結果を見てみる
user_id = 19
predict_rating_itembase = get_pred_ratings(user_id, similarity_matrix)
show_results(user_id, predict_rating_itembase)

# 意図的に未評価にした全箇所について評価
test = test_matrix.copy()
rmse = calc_RMSE(predict_rating_itembase[:, np.newaxis], test[user_id - 1][:, np.newaxis])
print()
print("RMSE")
print("\t", rmse)

# 意図的に未評価にした箇所の内のhigh rate部分のみについて評価
test *= blank_flag_matrix_high_rate
rmse = calc_RMSE(predict_rating_itembase[:, np.newaxis] * blank_flag_matrix_high_rate[user_id - 1], test[user_id - 1][:, np.newaxis])
print()
print("RMSE(only high rate)")
print("\t", rmse)

ジョーク 27
	真値 :	 6.625
	予測値 : 	 2.656982720146915
ジョーク 52
	真値 :	 -3.9375
	予測値 : 	 5.910785753777074
ジョーク 71
	真値 :	 -7.25
	予測値 : 	 7.3714206315182995
ジョーク 77
	真値 :	 9.65625
	予測値 : 	 -1.2210953291897118
ジョーク 105
	真値 :	 -6.875
	予測値 : 	 7.114475123048935

RMSE
	 2.0664654047171145

RMSE(only high rate)
	 0.9624899736181483


### 中心化(標準化)をしてみる
$Z = \frac{X − μ}{σ}$<br />
- Xはデータの集合,μはその平均値,σは標準偏差を表す
- 標準化した値のことをZスコアと呼んだりすることも


<img src="standarization.png" width="700">

ユーザによって全体的に高い評価を付けがちな人（甘めの人）と,全体的に低い評価を付けがちな人（辛めな人）がいる<br>
甘めの人がつけた評価4と，辛めの人がつけた評価2は，値は異なっていても実質的な意味合いには相違ないことがあるため，それを消したい

In [42]:
def zscore(x, axis = None):
    zscore = x.copy()
    mean_list = []
    
    for i in range(len(x)):
        xmean = np.mean(x[i], axis=axis, keepdims=True) # 平均
        xstd  = np.std(x[i], axis=axis, keepdims=True) # 標準偏差
        zscore[i] = (x[i] - xmean) / xstd
        mean_list.append(xmean)
        
    return zscore, mean_list

In [48]:
# 中心化を実際に行う
zscore_train_matrix, mean_list = zscore(train_matrix)
# 中心化を行なった結果からユーザ間の類似度を算出
zscore_similarity_matrix = cosine_similarity_matrix(zscore_train_matrix)

# 一応平均値は 0 で標準偏差は 1 になっていることを確認
print("平均　：　", np.mean(zscore_train_matrix), "標準偏差　：　", np.std(zscore_train_matrix))

# 対角を0に
np.fill_diagonal(zscore_similarity_matrix, 0)

print(zscore_similarity_matrix.shape)
zscore_similarity_matrix

平均　：　 -1.314506425376375e-19 標準偏差　：　 0.9999999999999994
(25045, 25045)


array([[ 0.00000000e+00,  3.35392355e-03, -4.17559339e-04, ...,
         2.24441242e-05,  8.43763254e-04,  6.00705558e-04],
       [ 3.35392355e-03,  0.00000000e+00, -1.44196341e-03, ...,
        -1.06820518e-04,  8.41256659e-04, -2.93226010e-05],
       [-4.17559339e-04, -1.44196341e-03,  0.00000000e+00, ...,
         5.82252860e-05,  6.94315600e-04,  1.15320606e-04],
       ...,
       [ 2.24441242e-05, -1.06820518e-04,  5.82252860e-05, ...,
         0.00000000e+00, -3.17509131e-04, -6.01780669e-05],
       [ 8.43763254e-04,  8.41256659e-04,  6.94315600e-04, ...,
        -3.17509131e-04,  0.00000000e+00,  1.24021241e-04],
       [ 6.00705558e-04, -2.93226010e-05,  1.15320606e-04, ...,
        -6.01780669e-05,  1.24021241e-04,  0.00000000e+00]])

In [35]:
# user_ID 19の人の予測結果を見てみる
user_id = 19
zscore_predict_rating_itembase = get_pred_ratings(user_id, zscore_similarity_matrix, mean_list)
show_results(user_id, zscore_predict_rating_itembase)

# 意図的に未評価にした全箇所について評価
test = test_matrix.copy()
rmse = calc_RMSE(zscore_predict_rating_itembase[:, np.newaxis], test[user_id - 1][:, np.newaxis])
print()
print("RMSE(normalized)")
print("\t", rmse)

# 意図的に未評価にした箇所の内のhigh rate部分のみについて評価
test *= blank_flag_matrix_high_rate
rmse = calc_RMSE(zscore_predict_rating_itembase[:, np.newaxis] * blank_flag_matrix_high_rate[user_id - 1], test[user_id - 1][:, np.newaxis])
print()
print("RMSE(normalized)(only high rate)")
print("\t", rmse)

ジョーク 27
	真値 :	 6.625
	予測値 : 	 5.288889913029861
ジョーク 52
	真値 :	 -3.9375
	予測値 : 	 6.175591646332428
ジョーク 71
	真値 :	 -7.25
	予測値 : 	 1.33250965622613
ジョーク 77
	真値 :	 9.65625
	予測値 : 	 5.411373385349652
ジョーク 105
	真値 :	 -6.875
	予測値 : 	 3.980536591008338

RMSE(normalized)
	 1.4458698349970585

RMSE(normalized)(only high rate)
	 0.9532849751829094


中心化前<br>
RMSE<br>
　　2.0664654047171145

RMSE(only high rate)<br>
　　0.9561512748601377

In [36]:
# 一応別の人も見てみる
# user_ID 122の人の予測結果を見てみる
user_id = 122
normal_predict_rating_userbase = get_pred_ratings(user_id, similarity_matrix)
show_results(user_id, normal_predict_rating_userbase)

# 意図的に未評価にした全箇所について評価
test = test_matrix.copy()
rmse = calc_RMSE(normal_predict_rating_userbase[:, np.newaxis], test_matrix[user_id - 1][:, np.newaxis])
print("RMSE")
print("\t", rmse)

zscore_predict_rating_userbase = get_pred_ratings(user_id, zscore_similarity_matrix, mean_list)
#show_results(user_id, zscore_predict_rating_userbase)

rmse = calc_RMSE(zscore_predict_rating_userbase[:, np.newaxis], test_matrix[user_id - 1][:, np.newaxis])
print("\t\t↓")
print("RMSE(normalized)")
print("\t", rmse)
print()

# 意図的に未評価にした箇所の内のhigh rate部分のみについて評価
test *= blank_flag_matrix_high_rate
rmse = calc_RMSE(normal_predict_rating_userbase[:, np.newaxis] * blank_flag_matrix_high_rate[user_id - 1], test[user_id - 1][:, np.newaxis])
print()
print("RMSE(only high rate)")
print("\t", rmse)

rmse = calc_RMSE(zscore_predict_rating_userbase[:, np.newaxis] * blank_flag_matrix_high_rate[user_id - 1], test[user_id - 1][:, np.newaxis])
print("\t\t↓")
print("RMSE(normalized)(only high rate)")
print("\t", rmse)

ジョーク 53
	真値 :	 6.96875
	予測値 : 	 -4.060354201942496
ジョーク 61
	真値 :	 4.78125
	予測値 : 	 7.3678968147381445
ジョーク 66
	真値 :	 3.9375
	予測値 : 	 3.53060059982618
ジョーク 81
	真値 :	 3.875
	予測値 : 	 4.052803126929854
ジョーク 111
	真値 :	 3.8125
	予測値 : 	 -3.6590300215860143
RMSE
	 1.1086112128207062
		↓
RMSE(normalized)
	 0.6334426518466176


RMSE(only high rate)
	 0.5755833332952057
		↓
RMSE(normalized)(only high rate)
	 0.5741601213446859


In [ ]:
# 中心化でどれだけの予測が性能向上したかを確認
better = 0
worse = 0

for i in range(train_matrix.shape[0]):
    user_id = i
    user_index = user_id - 1
    
    normal_predict_rating_itembase = get_pred_ratings(user_id, similarity_matrix)
    normal_rmse = calc_RMSE(normal_predict_rating_itembase[:, np.newaxis], test_matrix[user_index][:, np.newaxis])
    
    zscore_predict_rating_itembase = get_pred_ratings(user_id, zscore_similarity_matrix, mean_list)
    zscore_rmse = calc_RMSE(zscore_predict_rating_itembase[:, np.newaxis], test_matrix[user_index][:, np.newaxis])
    
    result = normal_rmse - zscore_rmse
    
    if result > 0:
        better += 1
    else:
        worse +=1
    
print("性能向上 : ", better)
print("性能悪化 : ", worse)
'''
性能向上 :  19614
性能悪化 :  5431
'''

In [37]:
# 中心化前の全ユーザの欠損部分の評価予測
user_base_result = []

for user_id in range(ratings.shape[0]):
    predict_rating_userbase = get_pred_ratings(user_id, similarity_matrix)
    user_base_result.append(predict_rating_userbase)
    
train = np.array(user_base_result)
test = test_matrix.copy()

# 評価予測の精度を確認
rmse = calc_RMSE(train, test)
print("RMSE")
print("\t", rmse)
'''
RMSE
	 1.6547369254853999
'''

RMSE
	 1.6547369254853999


In [38]:
# 中心化後の全ユーザの欠損部分の評価予測
user_base_result = []

for user_id in range(ratings.shape[0]):
    predict_rating_userbase = get_pred_ratings(user_id, zscore_similarity_matrix,  mean_list)
    user_base_result.append(predict_rating_userbase)
    
train = np.array(user_base_result)
test = test_matrix.copy()

# 評価予測の精度を確認
rmse = calc_RMSE(train, test)
print("RMSE(normalized)")
print("\t", rmse)
'''
RMSE(normalized)
	 1.4723042867865759
'''

RMSE(normalized)
	 1.4723042867865759
